# DeepEval: LLM & AI Agent Evaluation Framework

## 1. DeepEval 소개

**DeepEval**은 LLM 애플리케이션과 AI Agent를 평가하기 위한 오픈소스 프레임워크입니다.

### 왜 DeepEval인가?

LLM 기반 애플리케이션 개발 시 직면하는 문제:
- 답변 품질을 어떻게 측정할 것인가?
- 환각(Hallucination)을 어떻게 감지할 것인가?
- 충분한 테스트 데이터가 없다면?
- Agent 동작을 어떻게 평가할 것인가?

DeepEval은 이러한 문제를 해결하는 **평가 중심(Evaluation-First)** 프레임워크입니다.

---

## 2. DeepEval 핵심 기능

### 2.1. Evaluation Metrics (평가 메트릭)

DeepEval은 14가지 이상의 평가 메트릭을 제공합니다:

**RAG 평가 메트릭**
- **Faithfulness**: 답변이 컨텍스트에 근거하는가?
- **Answer Relevancy**: 답변이 질문과 관련 있는가?
- **Contextual Relevancy**: 컨텍스트가 질문과 관련 있는가?
- **Contextual Precision**: 컨텍스트가 정확한가?
- **Contextual Recall**: 필요한 컨텍스트를 모두 검색했는가?

**Agent 평가 메트릭**
- **Tool Correctness**: 도구를 올바르게 선택했는가?
- **Agent Goal Success**: Agent가 목표를 달성했는가?

**일반 메트릭**
- **Hallucination**: 환각이 있는가?
- **Toxicity**: 유해한 내용이 있는가?
- **Bias**: 편향이 있는가?

### 2.2. Synthesizer (데이터 생성)

**Golden Dataset이 부족할 때** 합성 데이터를 자동 생성합니다:
- 문서에서 질문-답변 쌍 생성
- Evolution 전략으로 질문 난이도 조정
- 품질 점수 자동 계산

### 2.3. CI/CD Integration

Pytest와 통합하여 **자동화된 평가 파이프라인** 구축 가능:
```python
@pytest.mark.parametrize("test_case", test_cases)
def test_llm_output(test_case):
    assert_test(test_case, metrics)
```

### 2.4. Tracing & Observability

Langfuse, Confident AI와 통합하여 **실시간 모니터링** 가능

---

## 3. DeepEval vs RAGAS 비교

| 기능 | DeepEval | RAGAS |
|------|----------|-------|
| **평가 메트릭** | 14+ (Agent 포함) | 8+ (RAG 중심) |
| **Synthesizer** | 7가지 Evolution 전략 | 간단한 생성 |
| **Agent 평가** | 전용 메트릭 제공 | 제한적 |
| **CI/CD 통합** | Pytest 네이티브 | 별도 작업 필요 |
| **커스터마이징** | 메트릭 직접 구현 가능 | 제한적 |
| **대시보드** | Confident AI | 없음 |

**선택 가이드**:
- **RAG 시스템 평가**: 둘 다 우수
- **Agent 평가**: DeepEval 권장
- **CI/CD 통합**: DeepEval 권장
- **간단한 시작**: RAGAS 권장

---

## 4. DeepEval 사용 시나리오

### 시나리오 1: RAG 시스템 평가
```python
from deepeval.metrics import FaithfulnessMetric
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input="질문",
    actual_output="답변",
    retrieval_context=["문서1", "문서2"]
)

metric = FaithfulnessMetric(threshold=0.7)
metric.measure(test_case)
print(metric.score)  # 0.85
```

### 시나리오 2: Agent 평가
```python
from deepeval.metrics import ToolCorrectnessMetric

metric = ToolCorrectnessMetric(threshold=0.8)
metric.measure(agent_test_case)
```

### 시나리오 3: Golden Dataset 생성
```python
from deepeval.synthesizer import Synthesizer

synthesizer = Synthesizer(model="gpt-4o")
goldens = synthesizer.generate_goldens_from_docs(
    document_chunks=chunks,
    max_goldens=100
)
```

---

## 5. 이 노트북에서 다루는 내용

이 노트북은 **DeepEval Synthesizer**에 집중합니다:

1. Evolution 전략 이해
2. 커스텀 가중치 설정
3. Golden Dataset 생성
4. 품질 필터링
5. 실제 평가 실행
6. Langfuse 통합

**학습 목표**:
- Synthesizer로 평가 데이터 자동 생성
- Evolution 전략으로 난이도 조절
- 생성된 데이터로 Agent 평가

---

## 참고 자료

- [DeepEval 공식 문서](https://docs.confident-ai.com/)
- [DeepEval GitHub](https://github.com/confident-ai/deepeval)
- [Agent 평가 가이드](https://deepeval.com/docs/getting-started-agents)
- [Synthesizer 가이드](https://docs.confident-ai.com/docs/evaluation-datasets-synthetic-data)

## 6. DeepEval 작동 원리

DeepEval은 **LLM을 Judge로 사용**하여 평가를 수행합니다:

1. **평가 시**: LLM이 답변을 분석하고 점수를 계산
2. **데이터 생성 시**: LLM이 문서에서 질문-답변 쌍을 생성

따라서 평가와 생성 모두에서 **LLM API 호출이 발생**합니다.

# 7. DeepEval Synthesizer: Evolution 전략

DeepEval의 가장 강력한 기능 중 하나는 **Synthesizer**입니다. Synthesizer는 문서에서 자동으로 Golden Dataset을 생성하며, **7가지 Evolution 전략**을 통해 질문의 난이도와 유형을 조절할 수 있습니다.

## Evolution 전략이란?

**Evolution 전략**은 기본 질문을 더 복잡하고 현실적인 질문으로 진화시키는 방법입니다. 이를 통해:
- 단순한 질문에서 복잡한 추론을 요구하는 질문으로 변환
- 다양한 난이도와 유형의 테스트 케이스 생성
- 실제 사용자 질문과 유사한 패턴 구현

---

## 7가지 Evolution 전략 상세 설명

### 1. REASONING (추론)
**목적**: 다단계 논리적 사고를 요구하는 질문 생성

**특징**:
- 여러 단계의 추론이 필요
- 조건부 논리 적용
- 인과관계 파악 요구

**예시**:
- **Before**: "비밀번호 재설정 절차는?"
- **After**: "Q1에 설정한 비밀번호가 90일 정책에 따라 만료되었고, 2FA 인증이 실패한 경우 비밀번호를 재설정하는 전체 절차는?"

---

### 2. MULTICONTEXT (다중 컨텍스트)
**목적**: 여러 문서의 정보를 통합하여 답변해야 하는 질문

**특징**:
- 2개 이상의 문서 참조
- 정보 통합 능력 테스트
- RAG 시스템의 문서 검색 정확도 평가

**예시**:
- **Before**: "VPN 문제 해결 방법은?"
- **After**: "비밀번호 정책과 VPN 인증 요구사항을 모두 고려했을 때, 'Invalid credentials' 오류를 해결하는 방법은?"

---

### 3. CONCRETIZING (구체화)
**목적**: 추상적 개념을 구체적 수치/예시로 변환

**특징**:
- 실제 숫자와 단위 사용
- 구체적인 시나리오 제시
- 정량적 분석 요구

**예시**:
- **Before**: "이메일 저장 용량 제한은?"
- **After**: "1,000개의 이메일(각 5MB)을 보관하려면 표준 메일박스 50GB 할당량으로 충분한가?"

---

### 4. CONSTRAINED (제약)
**목적**: 조건이나 제약 사항을 추가

**특징**:
- 시간/비용/권한 제약 추가
- 정책 준수 확인
- 예외 처리 능력 테스트

**예시**:
- **Before**: "소프트웨어 설치 방법은?"
- **After**: "예산 제약이 있고, 보안 팀 승인이 5일 이내에 필요하며, 관리자 권한 없이 소프트웨어를 설치하는 방법은?"

---

### 5. COMPARATIVE (비교)
**목적**: 여러 옵션을 비교 분석하도록 요구

**특징**:
- 2개 이상의 대안 제시
- 기준별 비교 요구
- 의사결정 지원 테스트

**예시**:
- **Before**: "원격 근무 지원 옵션은?"
- **After**: "라이브 챗과 전화 지원을 응답 시간, 비용, 해결률 측면에서 비교하면?"

---

### 6. HYPOTHETICAL (가상 시나리오)
**목적**: 예외 상황이나 가상 시나리오 적용

**특징**:
- "만약~라면" 질문
- 드문 상황 대응 테스트
- 위기 관리 능력 평가

**예시**:
- **Before**: "이메일 보안 모범 사례는?"
- **After**: "CEO 사칭 피싱 이메일을 받았을 때, 회사 정책에 따라 취해야 할 즉각적인 조치는?"

---

### 7. IN_BREADTH (범위 확장)
**목적**: 질문의 범위를 더 넓은 시스템/영향으로 확장

**특징**:
- 시스템 간 연쇄 영향 분석
- 전체적인 관점 요구
- 통합 시스템 이해 테스트

**예시**:
- **Before**: "VPN 연결 오류의 원인은?"
- **After**: "VPN 연결 오류가 이메일 동기화, 파일 공유, 화상 회의 시스템에 미치는 연쇄 영향은?"

---

## Evolution 전략 선택 가이드

| 전략 | 사용 시기 | Agent 평가 적합도 |
|------|-----------|-------------------|
| **REASONING** | 복잡한 의사결정 테스트 | 높음 |
| **MULTICONTEXT** | RAG 검색 품질 평가 | 중간 |
| **CONCRETIZING** | 정량적 분석 능력 | 중간 |
| **CONSTRAINED** | 정책 준수 확인 | 높음 |
| **COMPARATIVE** | 옵션 비교 능력 | 중간 |
| **HYPOTHETICAL** | 예외 처리 능력 | 높음 |
| **IN_BREADTH** | 시스템 이해도 | 낮음 |

---

## 참고 자료

- [DeepEval Synthesizer 공식 문서](https://docs.confident-ai.com/docs/evaluation-datasets-synthetic-data)
- [Evolution Strategies 상세 가이드](https://docs.confident-ai.com/docs/evaluation-datasets-synthetic-data#evolution-strategies)
- [Automatic Instruction Evolving for Large Language Models](https://www.themoonlight.io/ko/review/automatic-instruction-evolving-for-large-language-models)

In [1]:
import json
import os
from pathlib import Path
from typing import Any

from deepeval.dataset import EvaluationDataset, Golden
from deepeval.synthesizer import Synthesizer
from dotenv import load_dotenv

load_dotenv()

True

## 4. 기본 Synthesizer 사용

먼저 기본 설정으로 Synthesizer를 사용해봅니다.

In [ ]:
## 8. 기본 Synthesizer 사용

먼저 기본 설정으로 Synthesizer를 사용해봅니다.

**주의**: `document_chunks` 파라미터에 실제 문서를 제공해야 합니다. 빈 리스트로 테스트하면 에러가 발생할 수 있습니다.

In [ ]:
# 기본 설정으로 Golden Dataset 생성 (소규모 테스트)
basic_dataset = synthesizer.generate_goldens_from_docs(
    document_chunks=[],
    max_goldens=10,  # 10개만 생성
    num_evolutions=1,  # Evolution 1회 적용
    include_reasoning=True,  # 추론 과정 포함
)
basic_dataset

In [ ]:
# 생성된 Golden Dataset 확인

for i, golden in enumerate(basic_dataset.goldens[:3]):
    print(f"━━━ Golden {i + 1} ━━━")
    print(f"질문 (Input): {golden.input}")
    print(f"\n기대 답변 (Expected): {golden.expected_output[:200]}...")
    if golden.context:
        print(f"\n컨텍스트 수: {len(golden.context)}개")
    print()

## 9. Evolution 가중치 커스터마이징

비즈니스 요구사항에 맞게 Evolution 전략의 가중치를 조정합니다.

### 시나리오: IT Helpdesk 규정 준수 테스트

**목표**: 직원들이 회사 정책(비밀번호 정책, 보안 절차 등)을 잘 준수하는지 평가

**전략**:
- `CONSTRAINED` (제약) 가중치 높게: 정책 준수 확인
- `HYPOTHETICAL` (가상 시나리오): 예외 상황 대응
- `REASONING` (추론): 복잡한 정책 이해

In [2]:
# Evolution 가중치 설정
compliance_weights = {
    "REASONING": 0.15,  # 15%: 복잡한 추론
    "MULTICONTEXT": 0.05,  # 5%: 여러 문서 통합 (덜 중요)
    "CONCRETIZING": 0.10,  # 10%: 구체적 수치 확인
    "CONSTRAINED": 0.35,  # 35%: 제약/규정 준수 (가장 중요)
    "COMPARATIVE": 0.10,  # 10%: 옵션 비교
    "HYPOTHETICAL": 0.20,  # 20%: 예외 상황 (중요)
    "IN_BREADTH": 0.05,  # 5%: 범위 확장 (덜 중요)
}

print("규정 준수 테스트를 위한 Evolution 가중치:")
for strategy, weight in sorted(compliance_weights.items(), key=lambda x: x[1], reverse=True):
    print(f"  {strategy:15s}: {weight:5.1%}")

규정 준수 테스트를 위한 Evolution 가중치:
  CONSTRAINED    : 35.0%
  HYPOTHETICAL   : 20.0%
  REASONING      : 15.0%
  CONCRETIZING   : 10.0%
  COMPARATIVE    : 10.0%
  MULTICONTEXT   :  5.0%
  IN_BREADTH     :  5.0%


In [ ]:
# 커스터마이징된 가중치로 Golden Dataset 생성
compliance_dataset = synthesizer.generate_goldens_from_docs(
    document_chunks=[],
    max_goldens=50,  # 50개 생성
    num_evolutions=2,  # Evolution 2회 적용
    evolution_weights=compliance_weights,  # 커스텀 가중치
    include_reasoning=True,
)
compliance_dataset

## 10. 품질 필터링

생성된 Golden 중 품질이 낮은 것을 필터링합니다.

In [ ]:
import pandas as pd

quality_scores = []
for golden in compliance_dataset.goldens:
    score = golden.additional_metadata.get("synthetic_quality_score", 0)
    quality_scores.append(
        {
            "input": golden.input[:80] + "...",
            "quality_score": score,
            "evolutions": golden.additional_metadata.get("evolutions", []),
        }
    )

df_quality = pd.DataFrame(quality_scores)

print(df_quality["quality_score"].describe())
print(f"\n평균 품질 점수: {df_quality['quality_score'].mean():.3f}")

In [ ]:
# 품질 필터링 (0.7 이상만 유지)
QUALITY_THRESHOLD = 0.7

high_quality_goldens = [
    golden
    for golden in compliance_dataset.goldens
    if golden.additional_metadata.get("synthetic_quality_score", 0) >= QUALITY_THRESHOLD
]

print("품질 필터링 결과:")
print(f"  전체: {len(compliance_dataset.goldens)}개")
print(f"  필터링 후: {len(high_quality_goldens)}개")
print(f"  통과율: {len(high_quality_goldens) / len(compliance_dataset.goldens) * 100:.1f}%")

In [ ]:
import random

sample_size = max(1, int(len(high_quality_goldens) * 0.1))
review_samples = random.sample(high_quality_goldens, min(sample_size, 5))  # 최대 5개

print(f"\nHuman In the Loop 샘플링 ({len(review_samples)}개):\n")

for i, golden in enumerate(review_samples):
    print(f"━━━ 샘플 {i + 1} ━━━")
    print(f"질문: {golden.input}")
    print(f"기대 답변: {golden.expected_output[:150]}...")
    print(f"적용된 Evolution: {golden.additional_metadata.get('evolutions', [])}")
    print(f"품질 점수: {golden.additional_metadata.get('synthetic_quality_score', 0):.2f}")

## 11. 결과 분석 및 저장

생성된 Golden Dataset을 분석하고 저장합니다.

In [ ]:
from collections import Counter

all_evolutions = []
for golden in high_quality_goldens:
    evolutions = golden.additional_metadata.get("evolutions", [])
    all_evolutions.extend(evolutions)

evolution_counts = Counter(all_evolutions)

for strategy, count in evolution_counts.most_common():
    percentage = count / len(all_evolutions) * 100
    print(f"  {strategy:15s}: {count:3d}회 ({percentage:5.1f}%)")

In [ ]:
# JSONL 형식으로 저장
output_path = Path("generated_golden_dataset.jsonl")

with output_path.open("w", encoding="utf-8") as f:
    for golden in high_quality_goldens:
        record = {
            "question": golden.input,
            "ground_truth": golden.expected_output,
            "contexts": golden.context if golden.context else [],
            "metadata": {
                "quality_score": golden.additional_metadata.get("synthetic_quality_score", 0),
                "evolutions": golden.additional_metadata.get("evolutions", []),
                "source": "deepeval_synthesizer",
            },
        }
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

## 12. 생성된 Golden Dataset으로 실제 평가 실행

이제 생성한 Golden Dataset을 사용하여 **실제 Agent 시스템을 평가**합니다.

In [3]:
# DeepEval 평가 메트릭 import
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase
from langchain_openai import ChatOpenAI


In [ ]:
llm = ChatOpenAI(
    model="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=0,
)

In [ ]:
# 평가할 샘플 선정 (처음 5개)
test_samples = high_quality_goldens[:5]
test_samples

In [ ]:
# DeepEval 메트릭 초기화
faithfulness_metric = FaithfulnessMetric(model="openai/gpt-4.1", threshold=0.7, include_reason=True)
relevancy_metric = AnswerRelevancyMetric(model="openai/gpt-4.1", threshold=0.7, include_reason=True)

In [ ]:
# 실제 평가 실행
evaluation_results = []

for i, golden in enumerate(test_samples, 1):
    print(f"\n{'=' * 60}")
    print(f"평가 {i}/{len(test_samples)}")
    print(f"{'=' * 60}")

    # LLM으로 실제 답변 생성
    prompt = f"""다음 컨텍스트를 참고하여 질문에 답변하세요.

컨텍스트:
{chr(10).join(golden.context if golden.context else [])}

질문: {golden.input}

답변:"""

    actual_output = llm.invoke(prompt).content

    print(f"질문: {golden.input[:100]}...")
    print(f"\n생성된 답변: {actual_output[:150]}...")

    # LLMTestCase 생성
    test_case = LLMTestCase(
        input=golden.input,
        actual_output=actual_output,
        expected_output=golden.expected_output,
        retrieval_context=golden.context if golden.context else [],
    )

    # FaithfulnessMetric 평가
    try:
        faithfulness_metric.measure(test_case)
        faithfulness_score = faithfulness_metric.score
        faithfulness_reason = faithfulness_metric.reason
        print(f"\nFaithfulness: {faithfulness_score:.3f}")
        print(f" 이유: {faithfulness_reason[:100]}...")
    except Exception as e:
        print(f"Faithfulness 평가 실패: {e}")
        faithfulness_score = 0.0
        faithfulness_reason = str(e)

    # AnswerRelevancyMetric 평가
    try:
        relevancy_metric.measure(test_case)
        relevancy_score = relevancy_metric.score
        relevancy_reason = relevancy_metric.reason
        print(f"Answer Relevancy: {relevancy_score:.3f}")
        print(f" 이유: {relevancy_reason[:100]}...")
    except Exception as e:
        print(f"Answer Relevancy 평가 실패: {e}")
        relevancy_score = 0.0
        relevancy_reason = str(e)

    # 결과 저장
    evaluation_results.append(
        {
            "question": golden.input,
            "actual_output": actual_output,
            "expected_output": golden.expected_output,
            "faithfulness_score": faithfulness_score,
            "faithfulness_reason": faithfulness_reason,
            "relevancy_score": relevancy_score,
            "relevancy_reason": relevancy_reason,
            "contexts": golden.context if golden.context else [],
        }
    )

In [ ]:
# 평가 결과 분석
import pandas as pd

eval_df = pd.DataFrame(evaluation_results)

print("평가 결과 요약\n")
print(f"평균 Faithfulness: {eval_df['faithfulness_score'].mean():.3f}")
print(f"평균 Answer Relevancy: {eval_df['relevancy_score'].mean():.3f}")
print("\n통과율 (>0.7):")
print(
    f"  Faithfulness: {(eval_df['faithfulness_score'] > 0.7).sum()}/{len(eval_df)} ({(eval_df['faithfulness_score'] > 0.7).mean() * 100:.1f}%)"
)
print(
    f"  Answer Relevancy: {(eval_df['relevancy_score'] > 0.7).sum()}/{len(eval_df)} ({(eval_df['relevancy_score'] > 0.7).mean() * 100:.1f}%)"
)

# 결과 저장
eval_results_path = Path("evaluation_results.json")
with eval_results_path.open("w", encoding="utf-8") as f:
    json.dump(evaluation_results, f, ensure_ascii=False, indent=2)


## 13. Langfuse 업로드하기

생성된 Golden Dataset을 Langfuse에 업로드하여 대시보드에서 관리합니다.

In [ ]:
# Langfuse Client 초기화
from langfuse import Langfuse

langfuse_client = Langfuse(
    public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
    secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
    host=os.getenv("LANGFUSE_HOST", "http://localhost:3000"),
)

## 14. Agent 평가 메트릭 사용하기

이제 DeepEval의 **Agent 전용 메트릭**을 사용하여 AI Agent의 동작을 평가합니다.

### Agent 평가 메트릭 종류

1. **Tool Correctness**: Agent가 올바른 도구를 선택했는가?
2. **Plan Quality**: Agent의 계획이 목표 달성에 적합한가?
3. **Multi Turn MCP Use**: Multi-turn 대화에서 MCP(Model Context Protocol) 사용이 적절한가?
4. **Custom Metric**: 비즈니스 요구사항에 맞는 커스텀 평가 기준

---

### 참고 자료

- [Tool Correctness 문서](https://deepeval.com/docs/metrics-tool-correctness)
- [Plan Quality 문서](https://deepeval.com/docs/metrics-plan-quality)
- [Multi Turn MCP Use 문서](https://deepeval.com/docs/metrics-multi-turn-mcp-use)
- [Custom Metric 문서](https://deepeval.com/docs/metrics-conversational-dag)

### 14.1. Tool Correctness Metric

**Tool Correctness**는 Agent가 주어진 작업을 수행하기 위해 **올바른 도구를 선택했는지** 평가합니다.

**평가 기준**:
- 사용된 도구가 작업 목표와 일치하는가?
- 도구 호출 순서가 논리적인가?
- 불필요한 도구 호출이 있는가?

**사용 시나리오**:
- Multi-tool Agent 평가
- 도구 선택 정확도 측정
- Agent 최적화

In [ ]:
from deepeval.metrics import ToolCorrectnessMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams

# Tool Correctness 메트릭 초기화
tool_correctness_metric = ToolCorrectnessMetric(
    threshold=0.7, model="openai/gpt-4.1", include_reason=True
)

# Agent의 도구 사용 기록 (예시)
# 실제로는 LangGraph Agent 실행 후 tool_calls를 추출합니다
agent_tools_used = [
    {
        "name": "tavily_search",
        "input": {"query": "비밀번호 정책"},
        "output": "비밀번호는 최소 12자 이상이어야 합니다...",
    },
    {
        "name": "document_search",
        "input": {"query": "보안 가이드"},
        "output": "보안 가이드라인에 따르면...",
    },
]

# Test Case 생성
test_case = LLMTestCase(
    input="회사 비밀번호 정책과 보안 가이드라인을 알려주세요",
    actual_output="비밀번호는 최소 12자 이상이며, 대소문자, 숫자, 특수문자를 포함해야 합니다. 보안 가이드라인에 따라 90일마다 변경해야 합니다.",
    tools_used=agent_tools_used,
)

# 평가 실행
try:
    tool_correctness_metric.measure(test_case)

    print("Tool Correctness Metric 평가 결과")
    print("=" * 60)
    print(f"점수: {tool_correctness_metric.score:.3f}")
    print(f"통과 여부: {'통과' if tool_correctness_metric.is_successful() else '실패'}")
    print(f"\n평가 근거:")
    print(f"{tool_correctness_metric.reason}")

except Exception as e:
    print(f"Tool Correctness 평가 실패: {e}")
    import traceback

    traceback.print_exc()

### 14.2. Plan Quality Metric

**Plan Quality**는 Agent가 생성한 **실행 계획의 품질**을 평가합니다.

**평가 기준**:
- 계획이 목표 달성에 적합한가?
- 단계가 논리적으로 구성되었는가?
- 실행 가능한 계획인가?
- 불필요한 단계가 포함되어 있지 않은가?

**사용 시나리오**:
- Planning Agent 평가
- Multi-step 작업 계획 검증
- Agent 의사결정 품질 측정

In [ ]:
from deepeval.metrics import PlanQualityMetric

# Plan Quality 메트릭 초기화
plan_quality_metric = PlanQualityMetric(threshold=0.7, model="openai/gpt-4.1", include_reason=True)

# Agent가 생성한 실행 계획 (예시)
agent_plan = """
1. 사용자의 비밀번호 재설정 요청 확인
2. 사용자 신원 인증 (이메일 또는 2FA)
3. 비밀번호 정책 확인 (최소 12자, 대소문자, 숫자, 특수문자)
4. 새 비밀번호 생성 또는 사용자 입력 받기
5. 비밀번호 정책 준수 여부 검증
6. 데이터베이스에 해시된 비밀번호 저장
7. 사용자에게 완료 알림 전송
"""

# Test Case 생성
test_case = LLMTestCase(
    input="사용자 비밀번호를 안전하게 재설정하는 절차를 계획하세요",
    actual_output=agent_plan,
    # 목표를 명시하면 더 정확한 평가 가능
    expected_output="비밀번호 재설정은 신원 확인, 정책 검증, 안전한 저장을 포함해야 합니다",
)

# 평가 실행
try:
    plan_quality_metric.measure(test_case)

    print("Plan Quality Metric 평가 결과")
    print("=" * 60)
    print(f"점수: {plan_quality_metric.score:.3f}")
    print(f"통과 여부: {'통과' if plan_quality_metric.is_successful() else '실패'}")
    print(f"\n평가 근거:")
    print(f"{plan_quality_metric.reason}")

except Exception as e:
    print(f"Plan Quality 평가 실패: {e}")
    print("\nPlan Quality 메트릭은 DeepEval 최신 버전에서만 지원될 수 있습니다.")
    print("대안: Custom Metric을 사용하여 계획 품질을 평가할 수 있습니다.")

### 14.3. Custom Metric (커스텀 평가 기준)

DeepEval의 기본 메트릭으로 충분하지 않을 때, **Custom Metric**을 정의하여 비즈니스 요구사항에 맞는 평가 기준을 구현할 수 있습니다.

**사용 시나리오**:
- 도메인 특화 평가 (의료, 법률, 금융)
- 회사 고유 품질 기준
- 규정 준수 검증
- 대화 흐름 평가 (Conversational DAG)

**구현 방법**:
1. `BaseMetric` 상속
2. `measure()` 메서드 구현
3. 평가 로직 작성
4. 점수 및 이유 반환

In [ ]:
from deepeval.metrics import BaseMetric
from deepeval.test_case import LLMTestCase
from deepeval.scorer import Scorer


class ComplianceMetric(BaseMetric):
    """
    IT Helpdesk 규정 준수 평가 커스텀 메트릭

    평가 기준:
    1. 회사 정책 준수 여부
    2. 보안 절차 언급 여부
    3. 적절한 에스컬레이션 제안 여부
    """

    def __init__(self, threshold: float = 0.7, model: str = "openai/gpt-4.1"):
        self.threshold = threshold
        self.model = model
        self.scorer = Scorer(model=model)

    def measure(self, test_case: LLMTestCase) -> float:
        """규정 준수 점수 계산"""

        # 평가 프롬프트 정의
        evaluation_prompt = f"""
다음 답변이 IT Helpdesk 규정을 얼마나 잘 준수하는지 평가하세요.

평가 기준:
1. 회사 정책 준수 (0-0.4점)
   - 비밀번호 정책, 보안 절차 등 정책을 정확히 언급했는가?

2. 보안 의식 (0-0.3점)
   - 보안 위험을 인지하고 안전한 방법을 제시했는가?

3. 에스컬레이션 (0-0.3점)
   - 필요 시 IT 지원팀이나 관리자에게 문의하도록 안내했는가?

질문: {test_case.input}
답변: {test_case.actual_output}

점수 (0.0-1.0):
"""

        # LLM으로 평가 수행
        try:
            response = self.scorer.score(evaluation_prompt)

            # 점수 추출 (응답에서 숫자 파싱)
            import re

            match = re.search(r"(\d+\.?\d*)", response)
            if match:
                score = float(match.group(1))
                # 0-1 범위로 정규화
                if score > 1:
                    score = score / 10.0  # 10점 척도인 경우
                self.score = min(1.0, max(0.0, score))
            else:
                self.score = 0.0

            self.reason = response
            self.success = self.score >= self.threshold

        except Exception as e:
            print(f"평가 실패: {e}")
            self.score = 0.0
            self.reason = f"평가 실패: {str(e)}"
            self.success = False

        return self.score

    def is_successful(self) -> bool:
        """threshold 이상이면 성공"""
        return self.success

    @property
    def __name__(self):
        return "Compliance Metric"


# Custom Metric 사용 예시
compliance_metric = ComplianceMetric(threshold=0.7, model="openai/gpt-4.1")

# Test Case 생성
test_case = LLMTestCase(
    input="비밀번호를 잊어버렸는데 어떻게 재설정하나요?",
    actual_output="""
비밀번호 재설정 절차:
1. 로그인 페이지에서 '비밀번호 찾기' 클릭
2. 등록된 이메일로 재설정 링크 수신
3. 링크 클릭 후 새 비밀번호 입력
4. 회사 정책에 따라 최소 12자, 대소문자+숫자+특수문자 포함 필요
5. 문제가 계속되면 IT 지원팀(내선 1234)에 문의하세요
""",
)

# 평가 실행
try:
    score = compliance_metric.measure(test_case)

    print("Custom Compliance Metric 평가 결과")
    print("=" * 60)
    print(f"점수: {score:.3f}")
    print(f"통과 여부: {'통과' if compliance_metric.is_successful() else '실패'}")
    print(f"\n평가 근거:")
    print(f"{compliance_metric.reason[:300]}...")

except Exception as e:
    print(f"Custom Metric 평가 실패: {e}")
    import traceback

    traceback.print_exc()

## 15. 요약 및 다음 단계

### 학습한 내용

1. **DeepEval 소개**
   - LLM 및 AI Agent 평가 프레임워크
   - 14+ 평가 메트릭 제공
   - Synthesizer로 Golden Dataset 자동 생성

2. **Synthesizer 활용**
   - 7가지 Evolution 전략
   - 커스텀 가중치 설정
   - 품질 필터링 및 Human-in-the-Loop

3. **Agent 평가 메트릭**
   - Tool Correctness: 도구 선택 정확도
   - Plan Quality: 실행 계획 품질
   - Custom Metric: 비즈니스 요구사항 맞춤 평가

4. **프로덕션 통합**
   - Langfuse 연동
   - 평가 결과 저장 및 분석
   - CI/CD 파이프라인 구축

---

### 실습 과제

#### 기본 과제
1. 자신의 도메인에 맞는 Evolution 가중치 설정
2. Golden Dataset 100개 생성 및 품질 필터링
3. 3가지 이상의 메트릭으로 평가 실행

#### 심화 과제
1. Custom Metric 구현 (도메인 특화 평가 기준)
2. Multi-turn 대화 평가 파이프라인 구축
3. Langfuse 대시보드에서 평가 트렌드 분석

---

### 다음 단계

- **Session 5**: RAGAS vs DeepEval 심화 비교
- **Session 6**: Human-in-the-Loop 워크플로우
- **Session 7**: LLM-as-a-Judge 편향 완화
- **Session 8**: 프로덕션 모니터링 및 알림

---

### 참고 자료

**DeepEval 공식 문서**
- [Getting Started](https://docs.confident-ai.com/)
- [Metrics Overview](https://docs.confident-ai.com/docs/metrics-introduction)
- [Synthesizer Guide](https://docs.confident-ai.com/docs/evaluation-datasets-synthetic-data)

**Agent 평가**
- [Tool Correctness](https://deepeval.com/docs/metrics-tool-correctness)
- [Plan Quality](https://deepeval.com/docs/metrics-plan-quality)
- [Custom Metrics](https://deepeval.com/docs/metrics-conversational-dag)

**통합 가이드**
- [Langfuse Integration](https://docs.confident-ai.com/docs/integrations-langfuse)
- [CI/CD with Pytest](https://docs.confident-ai.com/docs/evaluation-test-cases)

---

**예상 소요 시간**: 120-150분  
**난이도**: 중급-고급  
**예상 비용**: $1.50-3.00 (실제 LLM 호출 시)  
**최종 업데이트**: 2025-11-14

In [ ]:
# Langfuse Dataset 생성 및 업로드
dataset_name = "deepeval_synthesizer_goldens"

# Dataset 아이템 생성
for i, golden in enumerate(high_quality_goldens[:10], 1):  # 처음 10개만 업로드
    try:
        langfuse_client.create_dataset_item(
            dataset_name=dataset_name,
            input={
                "question": golden.input,
                "contexts": golden.context if golden.context else [],
            },
            expected_output=golden.expected_output,
            metadata={
                "quality_score": golden.additional_metadata.get("synthetic_quality_score", 0),
                "evolutions": golden.additional_metadata.get("evolutions", []),
                "source": "deepeval_synthesizer",
            },
        )

        if i % 5 == 0:
            print(f" {i}개 업로드 완료")

    except Exception as e:
        print(f"  {i}번째 아이템 업로드 실패: {e}")

# Flush
langfuse_client.flush()